[View in Colaboratory](https://colab.research.google.com/github/tabbydoc/table_detection/blob/master/table_detection.ipynb)

### Подключаем гуглдиск

In [3]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpuv55mjh2/pubring.gpg' created
gpg: /tmp/tmpuv55mjh2/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [18]:
!ls "drive/Colab Notebooks/Tabby"

annotations  images


### Устанавливаем зависимости

In [27]:
!apt-get install protobuf-compiler
!pip install Cython
!pip install jupyter
!pip install matplotlib

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9ubuntu5).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [14]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Counting objects: 21560, done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 21560 (delta 0), reused 0 (delta 0), pack-reused 21553
Receiving objects: 100% (21560/21560), 558.36 MiB | 37.47 MiB/s, done.
Resolving deltas: 100% (12748/12748), done.
Checking out files: 100% (2670/2670), done.


In [30]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/models/research/slim
!protoc ./models/research/object_detection/protos/*.proto --python_out=.

object_detection/protos/grid_anchor_generator.proto: File not found.
object_detection/protos/ssd_anchor_generator.proto: File not found.
object_detection/protos/multiscale_anchor_generator.proto: File not found.
models/research/object_detection/protos/anchor_generator.proto: Import "object_detection/protos/grid_anchor_generator.proto" was not found or had errors.
models/research/object_detection/protos/anchor_generator.proto: Import "object_detection/protos/ssd_anchor_generator.proto" was not found or had errors.
models/research/object_detection/protos/anchor_generator.proto: Import "object_detection/protos/multiscale_anchor_generator.proto" was not found or had errors.
models/research/object_detection/protos/anchor_generator.proto:13:5: "GridAnchorGenerator" is not defined.
models/research/object_detection/protos/anchor_generator.proto:14:5: "SsdAnchorGenerator" is not defined.
models/research/object_detection/protos/anchor_generator.proto:15:5: "MultiscaleAnchorGenerator" is not defi

In [20]:
!ls models

AUTHORS     CONTRIBUTING.md    LICENSE	 README.md  samples    WORKSPACE
CODEOWNERS  ISSUE_TEMPLATE.md  official  research   tutorials


In [25]:
!ls models/research/object_detection/protos/

anchor_generator.proto	     losses.proto
argmax_matcher.proto	     matcher.proto
bipartite_matcher.proto      mean_stddev_box_coder.proto
box_coder.proto		     model.proto
box_predictor.proto	     multiscale_anchor_generator.proto
eval.proto		     optimizer.proto
faster_rcnn_box_coder.proto  pipeline.proto
faster_rcnn.proto	     post_processing.proto
graph_rewriter.proto	     preprocessor.proto
grid_anchor_generator.proto  region_similarity_calculator.proto
hyperparams.proto	     square_box_coder.proto
image_resizer.proto	     ssd_anchor_generator.proto
__init__.py		     ssd.proto
input_reader.proto	     string_int_label_map.proto
keypoint_box_coder.proto     train.proto
